In [13]:
# | default_exp routes.access_token

In [14]:
# | exporti
import datetime as dt
import time

from dataclasses import dataclass, field
import httpx

import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.get_data as gd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

# import domolibrary.client.Logger as lg

In [15]:
# | hide
import os
import pandas as pd

# User Access Tokens

In [16]:
# | export
@gd.route_function
async def get_access_tokens(
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class=None,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accesstokens"

    res = await gd.get_data(
        url=url,
        method="GET",
        auth=auth,
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    [
        token.update({"expires": dt.datetime.utcfromtimestamp(token["expires"] / 1000)})
        for token in res.response
    ]

    return res

In [17]:
#| eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_access_tokens(debug_api=False, auth=token_auth)

pd.DataFrame([r for r in res.response if r["ownerId"] == 1216550715])

warning this token has not been validated by who_am_i, run get_auth_token first


,id,name,ownerId,ownerName,ownerEmail,expires
0,186852,DL test 2024-03-20,1216550715,None,None,2024-06-18 19:04:57
1,186918,DL_test,1216550715,None,None,2024-06-21 16:46:38
2,186935,DL test 2024-03-23,1216550715,None,None,2024-06-21 17:35:40
3,186943,DL test 2024-03-23,1216550715,None,None,2024-06-21 17:48:21
4,186945,DL test 2024-03-23,1216550715,None,None,2024-06-21 17:48:29


In [18]:
# | exporti
def generate_expiration_unixtimestamp(
    duration_in_days: int = 90, debug_prn: bool = False
):

    today = dt.datetime.today()
    expiration_date = today + dt.timedelta(days=duration_in_days)

    if debug_prn:
        print(f"expiration_date is {duration_in_days} from today {expiration_date}")

    return int(time.mktime(expiration_date.timetuple()) * 1000)

In [19]:
generate_expiration_unixtimestamp(1, debug_prn=True)

expiration_date is 1 from today 2024-03-24 12:19:59.835208


1711304399000

In [20]:
# | export
class AccessToken_GenerationError(de.DomoError):
    def __init__(
        self,
        user_id,
        domo_instance,
        parent_class,
        function_name,
        message=None,
    ):
        super().__init__(
            domo_instance=domo_instance,
            message=message or f"failure to generate access_token for {user_id}",
            function_name=function_name,
            parent_class=parent_class,
        )


@gd.route_function
async def generate_access_token(
    auth: dmda.DomoAuth,
    token_name: str,
    user_id,
    duration_in_days: 90,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class=None,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accesstokens"

    expiration_timestamp = generate_expiration_unixtimestamp(
        duration_in_days=duration_in_days
    )

    body = {"name": token_name, "ownerId": user_id, "expires": expiration_timestamp}

    res = await gd.get_data(
        url=url,
        method="POST",
        body=body,
        auth=auth,
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if res.status == 400:
        raise AccessToken_GenerationError(
            user_id=user_id,
            message=f"unable to generate access_token for {user_id} || did you pass a valid user_id",
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    if not res.is_success or not res.response["token"]:
        raise AccessToken_GenerationError(
            user_id=user_id,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

In [21]:
#| eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res_generate_token = await generate_access_token(
    user_id=1216550715, duration_in_days=90, token_name="DL_test", auth=token_auth
)

res_generate_token

warning this token has not been validated by who_am_i, run get_auth_token first


ResponseGetData(status=200, response={'id': 186953, 'name': 'DL_test', 'ownerId': 1216550715, 'ownerName': '8:26 - go to sleep', 'ownerEmail': 'test4@domo.com', 'expires': 1718993999000, 'token': 'c44d85c0415c6fdcc22c62c55829b9c68313604f31e6d704'}, is_success=True, parent_class=None)

In [22]:
# | export


class AccessToken_RevokeError(de.DomoError):
    def __init__(
        self,
        access_token_id,
        domo_instance,
        parent_class,
        function_name,
    ):
        super().__init__(
            domo_instance=domo_instance,
            message=f"failure to revoke token {access_token_id}",
            function_name=function_name,
            parent_class=parent_class,
        )


@gd.route_function
async def revoke_access_token(
    auth: dmda.DomoAuth,
    access_token_id: int,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class=None,
    session: httpx.AsyncClient = None,
):
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accesstokens/{access_token_id}"

    res = await gd.get_data(
        url=url,
        method="DELETE",
        auth=auth,
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        session = session
    )

    if not res.is_success:
        raise AccessToken_RevokeError(
            access_token_id=access_token_id,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_Name,
        )

    return res

In [23]:
#| eval : false
access_token_id = res_generate_token.response["id"]

await revoke_access_token(
    auth=token_auth, access_token_id=access_token_id, debug_api=False
)

ResponseGetData(status=200, response='', is_success=True, parent_class=None)

In [24]:
# | hide
import nbdev

nbdev.nbdev_export("./auth_accesstoken.ipynb")